In [12]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

print("準備完了")


準備完了


In [13]:
class ThubanNaver:

    X_train, y_train, X_test, y_test = [], [], [], []
    index_to_word, word_to_index = {}, {}
    

    def __init__(self):
        print("생성")
        

    

    def set_init_words(self):
        # set word dictionary
        index_to_word = pd.read_csv('words.csv')
        self.index_to_word = {idx:word for idx, word in index_to_word['word'].items()}
        self.word_to_index = {word:idx for idx, word in self.index_to_word.items()}
        
    def set_init_datas(self):
        read_train = pd.read_csv('train.csv')
        read_test = pd.read_csv('test.csv')
        
        self.X_train = [self.get_encoded_sentence(text) for idx, text in read_train['text'].items()]
        self.X_test = [self.get_encoded_sentence(text) for idx, text in read_test['text'].items()]
        
        self.y_train = [label for idx, label in read_train['label'].items()]
        self.y_test = [label for idx, label in read_test['label'].items()]
        
    def save_datas_to_csv(self):
        
        # 데이터를 읽어봅시다. 
        train_txt = pd.read_table('ratings_train.txt')
        test_txt = pd.read_table('ratings_test.txt')
        # 重複のデータを削除, そしてNANを削除
        train_txt = train_txt.drop_duplicates('document').dropna()
        test_txt = test_txt.drop_duplicates('document').dropna()
        
        
        okt = Okt()
        
        stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
        
        #セーブされる配列
        train_texts = []
        test_texts = []
        clean_words = []
        

        # 単語辞書をcsvファイルにセーブために韓国語だけまたはtoken化する
        for i in range(0,2):
            if i == 0:
                read_x_datas = train_txt['document']
                save_x_datas = train_texts
            else:
                ml_x_datas = test_txt['document']
                save_x_datas = test_texts
                
            for idx, item in enumerate(read_x_datas.items()):
                item_text = re.compile('[가-힣]+').findall(item[1])
                results = okt.morphs(' '.join(item_text))
                stopwords_result = []
                for result in results:
                    if result not in stopwords: #単語たちのstopwordsを削除
                        stopwords_result.append(result)
                
                
                save_x_datas.append(' '.join(stopwords_result)) # trainまたはtestデータ
                clean_words.extend(stopwords_result) #辞書のcsvのため       
                # print("{} {}".format(results, idx))
                
                if idx == 99:
                    break

        #単語たちの重複を削除
        clean_words = set(clean_words)
        clean_words = list(clean_words)
        
        #
        clean_words.insert(0, "<UNUSED>")
        clean_words.insert(0, "<UNK>")
        clean_words.insert(0, "<BOS>")
        clean_words.insert(0, "<PAD>")
        
        
        # words.csvセーブ
        pd.DataFrame(clean_words, columns=['word']).to_csv('words.csv', encoding="utf-8")
        self.set_init_words() #下のtexts, labelsをセーブするため (classの変数 index_to_word, word_to_indexに 実際の値を入力)
        
        
        # trainとtestをcsvでセーブする
        train_labels = [result for result in train_txt['label']][:len(train_texts)]
        test_labels = [result for result in test_txt['label']][:len(test_texts)]

        train_df = pd.DataFrame({'text':train_texts, 'label':train_labels})
        test_df = pd.DataFrame({'text':train_texts, 'label':train_labels})
        
        pd.DataFrame(train_df).to_csv('train.csv', encoding="utf-8")
        pd.DataFrame(test_df).to_csv('test.csv', encoding="utf-8")
        self.set_init_datas() # classの変数 X_train, y_train, X_test, y_testに 実際の値を入力
        
        
    def load_csv_datas(self):
        self.set_init_words()
        self.set_init_datas()
        
        
    # 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
    # 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
    def get_encoded_sentence(self, sentence):
        return [self.word_to_index['<BOS>']]+[self.word_to_index[word] if word in self.word_to_index else self.word_to_index['<UNK>'] for word in sentence.split()]

    # 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
    def get_encoded_sentences(self, sentences):
        return [self.get_encoded_sentence(sentence) for sentence in sentences]
    
    # 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
    def get_decoded_sentence(self, encoded_sentence):
        return ' '.join(self.index_to_word[index] if index in self.index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

    # 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
    def get_decoded_sentences(self, encoded_sentences):
        return [self.get_decoded_sentence(encoded_sentence, self.index_to_word) for encoded_sentence in encoded_sentences]

TN = ThubanNaver()

생성


In [14]:
TN.save_datas_to_csv()
print("データセーブ完了")

データセーブ完了


In [4]:
# すでにTN.save_datas_to_csv()をして words.csv, x_test.csv, x_train.csv ファイルが居る場合は下の関数を利用してデータを読み込みする
TN.set_init_words()
TN.set_init_datas()
print("データロード完了")


In [15]:
print(TN.get_decoded_sentence(TN.X_train[0]))
print(TN.y_train[0])
print()
print(TN.get_decoded_sentence(TN.X_train[1]))
print(TN.y_train[1])
print()

아 더빙 진짜 짜증나네요 목소리
0

흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나
1



In [ ]:
TN.get_encoded_sentence("먼가 많이 이상해 그렇게 생각하지 않아?")

In [ ]:
okt = Okt()
print(okt.morphs("안녕 하세요"))

In [ ]:
import platform 
print(platform.architecture())
